In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# hyper parameters
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST dataset(image and labels)

train_dataset = torchvision.datasets.MNIST(root = '../../data',
                                         train = True,
                                         transform=transforms.ToTensor(),
                                         download=True
                                         )

test_dataset = torchvision.datasets.MNIST(root = '../../data',
                                         train = False,
                                         transform=transforms.ToTensor(),
                                         )

# data loader(input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [7]:
# Convolutional Neural Network with two convolutional layers

class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = 5, stride = 1, padding = 2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size = 5, stride = 1, padding = 2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)        
        )
        self.fc = nn.Linear(7*7*32, num_classes)
        
        
        
        
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

In [8]:
# loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [10]:
# train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2292
Epoch [1/5], Step [200/600], Loss: 0.0839
Epoch [1/5], Step [300/600], Loss: 0.1572
Epoch [1/5], Step [400/600], Loss: 0.0953
Epoch [1/5], Step [500/600], Loss: 0.0319
Epoch [1/5], Step [600/600], Loss: 0.0222
Epoch [2/5], Step [100/600], Loss: 0.0503
Epoch [2/5], Step [200/600], Loss: 0.0453
Epoch [2/5], Step [300/600], Loss: 0.0548
Epoch [2/5], Step [400/600], Loss: 0.0484
Epoch [2/5], Step [500/600], Loss: 0.0267
Epoch [2/5], Step [600/600], Loss: 0.0183
Epoch [3/5], Step [100/600], Loss: 0.0367
Epoch [3/5], Step [200/600], Loss: 0.0664
Epoch [3/5], Step [300/600], Loss: 0.0424
Epoch [3/5], Step [400/600], Loss: 0.0341
Epoch [3/5], Step [500/600], Loss: 0.0319
Epoch [3/5], Step [600/600], Loss: 0.0403
Epoch [4/5], Step [100/600], Loss: 0.0261
Epoch [4/5], Step [200/600], Loss: 0.0035
Epoch [4/5], Step [300/600], Loss: 0.0303
Epoch [4/5], Step [400/600], Loss: 0.0340
Epoch [4/5], Step [500/600], Loss: 0.0750
Epoch [4/5], Step [600/600], Loss:

In [16]:
# test the model
# in test phase, don't compute gradients for memory efficiency

with torch.no_grad():
    correct = 0.0
    total = 0.0
    for images, labels in test_loader:
        # images = images.reshape(-1, 28*28)
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        
    print('Accuracy of the model on the 10000 test images: {} %'.format(100.0 * correct / total))
    



Accuracy of the model on the 10000 test images: 99 %


In [13]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

In [14]:
torch.save(model.state_dict(), 'Convolutional Neural Network.ckpt')